In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression,\
                                RidgeCV, LassoCV, LogisticRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder
from sklearn.metrics import r2_score, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.decomposition import PCA

[BOOKMARK](#BOOKMARK)

### Functions

In [2]:
results = pd.DataFrame(columns = ['Model', 'Train Score', 'Val Score', 'X Val Score', 'RMSE Train', 'RMSE Val'])

def update_df(model, model_name):
    '''fn updates a dataframe for quick reference of R squared scores and RMSE'''
    import warnings
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        
        train_score = model.score(Xs_train, y_train)
        val_score = model.score(Xs_test, y_test)
        x_val_score = cross_val_score(model, Xs_train, y_train).mean()
        rmse1 = mean_squared_error(y_train, lr.predict(Xs_train))**0.5
        rmse2 = mean_squared_error(y_test, lr.predict(Xs_test))**0.5
        
        results.loc[len(results.index)] = [model_name, train_score, val_score, x_val_score, rmse1, rmse2] 
        
    return results

### Read in data

In [3]:
df = pd.read_csv('./cleaned_data_descriptive/combined_df.csv',index_col=[0])

In [4]:
df.head()

,county,sex,xiang,q010,q015,q016,q022,q023,q024,q025,...,u024,u025,u026,u027,u028,u029,u030,u031,u032,u033
0,AA,F,1,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AA,F,2,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AA,F,3,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AA,T,3,40.2,60.0,70.0,3.3,22.4,41.9,35.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AB,F,1,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.shape

(275, 296)

In [6]:
df[df['sex'] == 'T'].shape

(69, 296)

In [7]:
ss = StandardScaler()

In [8]:
# df.columns.to_list() # sanity check

In [9]:
clean_df = df[(df['sex'] == 'T') & (df['xiang'] == 3)]\
.dropna(subset='m005_ALL35_69').dropna(axis = 'columns')

In [10]:
clean_df['m005_ALL35_69']

3      11.29
7      11.82
11     10.55
15     16.19
19     13.59
       ...  
259    17.05
263    13.29
266    15.63
270    18.88
274    17.95
Name: m005_ALL35_69, Length: 66, dtype: float64

In [11]:
clean_df.shape

(66, 201)

In [12]:
clean_df.describe()

,xiang,q010,q015,q016,q022,q023,q024,q025,q037,q038,...,r017,r018,r019,r020,r021,r022,r023,r024,r025,r026
count,66.0,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,...,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000
mean,3.0,75.156061,59.174242,72.848485,3.065152,29.019697,45.112121,24.787879,2112.939394,1004.287879,...,0.911818,0.855152,8.555152,0.394091,0.555303,3.926212,9.411364,0.537727,1.667424,11.307576
std,0.0,19.578026,19.377126,16.551544,2.691210,12.444637,9.117178,9.574333,1013.991153,832.391730,...,0.385239,0.739628,3.392067,0.313894,0.332858,1.557350,1.296920,0.093936,0.266373,2.896327
min,3.0,0.000000,13.000000,19.000000,0.000000,4.700000,19.400000,9.300000,691.000000,63.000000,...,0.280000,0.080000,3.730000,0.040000,0.130000,1.190000,6.540000,0.360000,1.170000,7.500000
25%,3.0,71.775000,45.625000,62.500000,1.600000,20.150000,38.900000,18.225000,1451.250000,418.750000,...,0.595000,0.312500,5.700000,0.200000,0.340000,2.625000,8.507500,0.490000,1.450000,9.200000
50%,3.0,81.950000,58.250000,75.000000,2.350000,28.350000,45.600000,23.400000,1983.000000,706.500000,...,0.830000,0.580000,7.700000,0.265000,0.465000,4.030000,9.300000,0.530000,1.625000,10.550000
75%,3.0,88.625000,72.375000,84.500000,4.375000,35.475000,51.675000,32.025000,2505.500000,1350.250000,...,1.250000,1.185000,11.312500,0.605000,0.682500,5.250000,10.060000,0.560000,1.817500,12.075000
max,3.0,96.700000,95.000000,100.000000,12.500000,63.000000,64.800000,49.600000,7238.000000,3925.000000,...,1.650000,2.720000,17.320000,1.830000,1.980000,6.460000,12.780000,0.780000,2.350000,19.600000


### Linear Regression

Linear Regression makes the most sense to do since all of our data is numeric albeit on different numeric scales. We can use standard scale and go from there.

In [13]:
X = clean_df.drop(columns = ['county', 'sex', 'xiang','m005_ALL35_69'])
y = clean_df['m005_ALL35_69']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=33)

We're suffering from dimensionality here; we have too many features. We need to figure out which ones to focus on; trying to limit to 10 features for our 69 or so rows. Lasso and/or Ridge could help us out here.

In [14]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((49, 197), (17, 197), (49,), (17,))

Standard scaler to get everything on the same page.

In [15]:
Xs_train = ss.fit_transform(X_train)
Xs_test = ss.transform(X_test)

In [16]:
lr = LinearRegression()

In [17]:
lr.fit(Xs_train, y_train)

LinearRegression()

In [18]:
update_df(lr, 'm005_ALL_35-69.LinReg')

,Model,Train Score,Val Score,X Val Score,RMSE Train,RMSE Val
0,m005_ALL_35-69.LinReg,1.0,0.88294,0.616316,8.404957e-15,1.206978


In [19]:
ridge = RidgeCV(alphas = np.logspace(0,2,100)) # 

In [20]:
lasso = LassoCV(alphas = np.arange(0.001, 10, 1))

In [21]:
ridge.fit(Xs_train, y_train)

RidgeCV(alphas=array([  1.        ,   1.04761575,   1.09749877,   1.149757  ,
         1.20450354,   1.26185688,   1.32194115,   1.38488637,
         1.45082878,   1.51991108,   1.59228279,   1.66810054,
         1.7475284 ,   1.83073828,   1.91791026,   2.009233  ,
         2.10490414,   2.20513074,   2.3101297 ,   2.42012826,
         2.53536449,   2.65608778,   2.7825594 ,   2.91505306,
         3.05385551,   3.19926714,   3.35160265,   3.51119173,
         3.67837977,   3.85352859,   4.03701726,   4....
        23.64489413,  24.77076356,  25.95024211,  27.18588243,
        28.48035868,  29.8364724 ,  31.2571585 ,  32.74549163,
        34.30469286,  35.93813664,  37.64935807,  39.44206059,
        41.320124  ,  43.28761281,  45.34878508,  47.50810162,
        49.77023564,  52.14008288,  54.62277218,  57.22367659,
        59.94842503,  62.80291442,  65.79332247,  68.92612104,
        72.20809018,  75.64633276,  79.24828984,  83.02175681,
        86.97490026,  91.11627561,  95.45484567, 100.        ]))

In [22]:
ridge.alpha_

1.0

In [23]:
update_df(ridge, "m005_ALL_35-69.L2-1")

,Model,Train Score,Val Score,X Val Score,RMSE Train,RMSE Val
0,m005_ALL_35-69.LinReg,1.000000,0.882940,0.616316,8.404957e-15,1.206978
1,m005_ALL_35-69.L2-1,0.999965,0.881611,0.614525,8.404957e-15,1.206978


In [24]:
lasso.fit(Xs_train, y_train)

LassoCV(alphas=array([1.000e-03, 1.001e+00, 2.001e+00, 3.001e+00, 4.001e+00, 5.001e+00,
       6.001e+00, 7.001e+00, 8.001e+00, 9.001e+00]))

In [25]:
update_df(lasso, "m005_ALL_35-69.L1-1")

,Model,Train Score,Val Score,X Val Score,RMSE Train,RMSE Val
0,m005_ALL_35-69.LinReg,1.000000,0.882940,0.616316,8.404957e-15,1.206978
1,m005_ALL_35-69.L2-1,0.999965,0.881611,0.614525,8.404957e-15,1.206978
2,m005_ALL_35-69.L1-1,0.999998,0.999996,0.999285,8.404957e-15,1.206978


In [26]:
importance = np.abs(lasso.coef_)
features = X.columns
np.array(features)[importance>0]

array(['q022', 'q096', 'q102', 'q105', 'q121', 'q122', 'q126', 'q152',
       'd038', 'm008_MEDICALc', 'm010_NONMEDc', 'p027', 'p034', 'p042',
       'r002', 'r016', 'r026'], dtype=object)

In [27]:
# np.array(features)
# importance
feat_imp = pd.DataFrame({'features':features, 'importance':importance}, columns=['features','importance'])

It'd be nice to be able to pull the code descriptions without searching the whole sheet one at a time:

In [28]:
with open('./data/CHNAME.txt', 'r') as file:
    data = file.read()
    rows = data.split('\n')

descriptions = pd.DataFrame(rows)

In [29]:
# descriptions.head()
# return every other row
text_df = descriptions.iloc[::2,:]
text_df['code'] = text_df[0].str.split(expand=True).iloc[:,0]
text_df.head()

/var/folders/2f/m4z0nfhn5px0p2mdppmmrlxr0000gn/T/ipykernel_31825/2593100514.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_df['code'] = text_df[0].str.split(expand=True).iloc[:,0]


,0,code
0,M001 ALL0-4 mortality ALL CAUSES AGE 0-4 (...,M001
2,M002 ALL5-14 mortality ALL CAUSES AGE 5-14 ...,M002
4,M003 ALL15-34 mortality ALL CAUSES AGE 15-34...,M003
6,M004 ALL0-34 mortality ALL CAUSES AGE 0-34 ...,M004
8,M005 ALL35-69 mortality ALL CAUSES AGE 35-69...,M005


In [30]:
text_df.rename(columns={0:'description'}, inplace=True)

/var/folders/2f/m4z0nfhn5px0p2mdppmmrlxr0000gn/T/ipykernel_31825/360386991.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_df.rename(columns={0:'description'}, inplace=True)


In [31]:
text_df['code'] = text_df['code'].str.lower()

/var/folders/2f/m4z0nfhn5px0p2mdppmmrlxr0000gn/T/ipykernel_31825/2186823998.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_df['code'] = text_df['code'].str.lower()


In [32]:
text_df.head()

,description,code
0,M001 ALL0-4 mortality ALL CAUSES AGE 0-4 (...,m001
2,M002 ALL5-14 mortality ALL CAUSES AGE 5-14 ...,m002
4,M003 ALL15-34 mortality ALL CAUSES AGE 15-34...,m003
6,M004 ALL0-34 mortality ALL CAUSES AGE 0-34 ...,m004
8,M005 ALL35-69 mortality ALL CAUSES AGE 35-69...,m005


In [33]:
feat_imp = feat_imp.merge(text_df, left_on='features', right_on='code', how='left').drop(columns = ['code'])

In [34]:
pd.set_option('max_colwidth', 150) # so I can see the full descriptions

In [35]:
feat_imp[feat_imp['importance']>0]

,features,importance,description
3,q022,0.000505,Q022 dEDUCATED questionnaire PERCENTAGE WHO ARE WELL-EDUCATED
23,q096,0.000027,Q096 dMALARIA questionnaire PERCENTAGE WITH HISTORY OF MALARIA DIAGNOSIS
29,q102,0.000886,Q102 dPHLEGMw questionnaire PERCENTAGE WHO COUGH UP PHLEGM MOST MORNINGS IN WINTER
32,q105,0.000454,Q105 dPHLEGMyr questionnaire NUMBER OF YEARS TROUBLED BY PHLEGM (years)
40,q121,0.000025,Q121 dANTIBIOT questionnaire PERCENTAGE USED MAINLY WESTERN ANTIBIOTICS DURING PAST 6 MONTHS
41,q122,0.000499,Q122 dANTACID questionnaire PERCENTAGE USED MAINLY WESTERN ANTACIDS DURING PAST 6 MONTHS
45,q126,0.000855,Q126 dWTLOSS questionnaire PERCENTAGE WHO LOST WEIGHT DURING FOOD SHORTAGE
61,q152,0.000007,Q152 dWINE questionnaire PERCENTAGE WHO HAVE EVER DRUNK WINE 3 OR MORE DAYS A WEEK FOR 6 MONTHS
107,d038,0.000116,"D038 WHTFLOUR diet survey WHEAT FLOUR INTAKE (g/day/reference man, air-dry basis)"
127,m008_MEDICALc,2.879213,NaN


In [36]:
clean_df['m008_MEDICALc'].describe()

count    66.000000
mean     12.991667
std       3.039937
min       8.010000
25%      10.467500
50%      13.040000
75%      14.940000
max      21.230000
Name: m008_MEDICALc, dtype: float64

We don't want other mortality reasons to affect our predictors. We will remove them from the feature list moving forward.

In [37]:
drop_columns = ['county', 'sex', 'xiang','m010_NONMEDc','m005_ALL35_69']

In [38]:
X = clean_df.drop(columns = drop_columns)
y = clean_df['m005_ALL35_69']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=33)

In [39]:
Xs_train = ss.fit_transform(X_train)
Xs_test = ss.transform(X_test)

In [40]:
lr.fit(Xs_train, y_train)
lasso.fit(Xs_train, y_train)
update_df(lasso, "m005_ALL_35-69.L1-2")

/Users/colby/miniforge3/envs/tfm1/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.310e-02, tolerance: 4.664e-02
  model = cd_fast.enet_coordinate_descent(


,Model,Train Score,Val Score,X Val Score,RMSE Train,RMSE Val
0,m005_ALL_35-69.LinReg,1.000000,0.882940,0.616316,8.404957e-15,1.206978
1,m005_ALL_35-69.L2-1,0.999965,0.881611,0.614525,8.404957e-15,1.206978
2,m005_ALL_35-69.L1-1,0.999998,0.999996,0.999285,8.404957e-15,1.206978
3,m005_ALL_35-69.L1-2,0.999989,0.988186,0.931555,5.548124e-15,1.264333


Great scores for Lasso 2 overall. We can use this model's settings as a springboard for the other mortalities.

## BOOKMARK

In [41]:
# clean_df.columns.to_list()
 # All the mortality columns
 # 'm005_ALL35_69',
 # 'm008_MEDICALc',
 # 'm065_STROKEc',
 # 'm023_ALLCAc',
 # 'm059_ALLVASCc',
 # 'm028_OESOPHCAc',
 # 'm072_COPDc',
 # 'm010_NONMEDc',

In [42]:
features_dropped = ['county', 'sex', 'xiang','m010_NONMEDc',\
                    'm005_ALL35_69', 'm008_MEDICALc', 'm065_STROKEc',\
                   'm023_ALLCAc', 'm059_ALLVASCc', 'm028_OESOPHCAc', 'm072_COPDc',\
                   'm072_COPDc']

mortality_feat = ['m010_NONMEDc', 'm005_ALL35_69', 'm008_MEDICALc', 'm065_STROKEc',\
                   'm023_ALLCAc', 'm059_ALLVASCc', 'm028_OESOPHCAc', 'm072_COPDc']

In [49]:
# Creating one code block to take care of all the models in one go:
lr = LinearRegression()
ss = StandardScaler()
lasso = LassoCV(alphas = np.arange(0.001, 10, 1))

imp_features = pd.DataFrame()

# looping over each target
for i in mortality_feat:
    # suppressing the convergence warnings if any
    import warnings
    warnings.filterwarnings("ignore")

    X = clean_df.drop(columns = features_dropped)
    y = clean_df[i]
    # train test split ('X before y, train before test' -James 2022)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=33)
    
    ss.fit(X_train)
    Xs_train = ss.transform(X_train)
    Xs_test = ss.transform(X_test)
    
    lr.fit(Xs_train, y_train)
    lasso.fit(Xs_train, y_train)
    
    mortal = i

    update_df(lasso, mortal)
    
    # creating a df to get feature importance out
    feature_imp = pd.Series(lasso.coef_, index = X.columns)
    feature_imp = feature_imp.sort_values(ascending=False)
    
    imp_features[i] = feature_imp


In [50]:
results

,Model,Train Score,Val Score,X Val Score,RMSE Train,RMSE Val
0,m005_ALL_35-69.LinReg,1.000000,0.882940,0.616316,8.404957e-15,1.206978
1,m005_ALL_35-69.L2-1,0.999965,0.881611,0.614525,8.404957e-15,1.206978
2,m005_ALL_35-69.L1-1,0.999998,0.999996,0.999285,8.404957e-15,1.206978
3,m005_ALL_35-69.L1-2,0.999989,0.988186,0.931555,5.548124e-15,1.264333
4,m010_NONMEDc,0.482118,0.169658,-0.124616,1.332323e-13,44.534195
5,m005_ALL35_69,0.999993,0.056338,0.070911,7.825686e-15,2.689775
6,m008_MEDICALc,0.999991,0.107569,0.160962,6.704399e-15,2.591834
7,m065_STROKEc,0.770332,0.290192,0.009490,2.591966e-13,130.003377
8,m023_ALLCAc,0.999981,-0.538021,-0.466490,5.398386e-15,1.862238
9,m059_ALLVASCc,0.999958,0.138958,-0.104030,3.350999e-15,1.742879


In [68]:
imp_features['m010_NONMEDc'][imp_features['m010_NONMEDc'] != 0].to_frame().sort_values(by='m010_NONMEDc', ascending=False)

,m010_NONMEDc
q153,7.084762
q023,5.215087
p025,4.552176
q105,4.505004
d030,3.514923
r024,2.796055
r016,2.492005
d028,1.967320
d044,0.017837
q108,-0.216117


In [69]:
imp_features['m005_ALL35_69'][imp_features['m005_ALL35_69'] != 0].to_frame().sort_values(by='m005_ALL35_69', ascending=False)

,m005_ALL35_69
q099,1.352670
p043,0.726593
d044,0.658357
d042,0.482376
q171,0.454607
...,...
q155,-0.300691
q040,-0.410812
p009,-0.545873
p038,-0.673292


In [70]:
imp_features['m008_MEDICALc'][imp_features['m008_MEDICALc'] != 0].to_frame().sort_values(by='m008_MEDICALc', ascending=False)

,m008_MEDICALc
q099,1.446953
p043,0.578106
d044,0.488523
d042,0.484209
q125,0.420315
...,...
p009,-0.368200
q010,-0.398946
p002,-0.489826
p027,-0.562347


In [71]:
imp_features['m065_STROKEc'][imp_features['m065_STROKEc'] != 0].to_frame().sort_values(by='m065_STROKEc', ascending=False)

,m065_STROKEc
q100,26.612120
q158,17.366433
q106,17.271656
r024,12.302131
r013,7.621683
p043,6.231466
q137,6.158920
q109,1.866571
r012,0.316645
q099,0.217354


In [72]:
imp_features['m023_ALLCAc'][imp_features['m023_ALLCAc'] != 0].to_frame().sort_values(by='m023_ALLCAc', ascending=False)

,m023_ALLCAc
q099,0.683157
q125,0.576475
p008,0.483848
d013,0.475164
q039,0.453919
...,...
q152,-0.308406
q144,-0.308743
q147,-0.352945
p038,-0.630836


In [73]:
imp_features['m059_ALLVASCc'][imp_features['m059_ALLVASCc'] != 0].to_frame().sort_values(by='m059_ALLVASCc', ascending=False)

,m059_ALLVASCc
q099,0.400425
q158,0.376016
d042,0.315672
p043,0.229516
q037,0.221994
d043,0.159212
q176,0.150553
r022,0.147163
q100,0.142196
q106,0.135022


In [74]:
imp_features['m028_OESOPHCAc'][imp_features['m028_OESOPHCAc'] != 0].to_frame().sort_values(by='m028_OESOPHCAc', ascending=False)

,m028_OESOPHCAc
p042,27.256861
p008,15.778510
d013,12.768285
q125,10.214094
p006,4.894704
q023,4.606957
g003,3.043453
q120,2.931746
q039,2.595203
q122,2.317649


In [78]:
imp_features['m072_COPDc'][imp_features['m072_COPDc'] > 0].to_frame().sort_values(by='m072_COPDc', ascending=False)

,m072_COPDc
q106,29.647219
q128,20.350247
r024,18.650919
d028,15.993840
p010,15.980178
d042,14.556566
d040,13.283286
q165,12.236443
d041,8.882953
p043,8.726487
